In [89]:
import pandas as pd
import numpy as np
import random
import os
from sklearn.preprocessing import LabelEncoder
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')

In [90]:
import numpy as np
from supervised import AutoML

In [91]:
train_df['사고일시'] = pd.to_datetime(train_df['사고일시'])

In [92]:
test_df['사고일시'] = pd.to_datetime(test_df['사고일시'])

# 군과 구를 나눠줍니다

In [93]:
train_df['군'] = ''
train_df['구'] = ''

test_df['군'] = ''
test_df['구'] = ''

# 주소 분리 및 군과 구 열에 저장
for i in range(0, len(train_df)):
    address = train_df['시군구'][i]
    elements = address.split(" ")
    if len(elements) == 3:
        city = elements[0]
        county = elements[1]
        district = elements[2]
        train_df.at[i, '군'] = county
        train_df.at[i, '구'] = district
    else:
        print(f"잘못된 주소 형식입니다: {address}")

# 결과 출력
print(train_df)

                   ID                사고일시   요일 기상상태              시군구  \
0      ACCIDENT_00000 2019-01-01 00:00:00  화요일   맑음     대구광역시 중구 대신동   
1      ACCIDENT_00001 2019-01-01 00:00:00  화요일   흐림    대구광역시 달서구 감삼동   
2      ACCIDENT_00002 2019-01-01 01:00:00  화요일   맑음    대구광역시 수성구 두산동   
3      ACCIDENT_00003 2019-01-01 02:00:00  화요일   맑음     대구광역시 북구 복현동   
4      ACCIDENT_00004 2019-01-01 04:00:00  화요일   맑음     대구광역시 동구 신암동   
...               ...                 ...  ...  ...              ...   
39604  ACCIDENT_39604 2021-12-31 19:00:00  금요일   맑음  대구광역시 수성구 수성동3가   
39605  ACCIDENT_39605 2021-12-31 19:00:00  금요일   맑음    대구광역시 달서구 상인동   
39606  ACCIDENT_39606 2021-12-31 21:00:00  금요일   맑음    대구광역시 달서구 월성동   
39607  ACCIDENT_39607 2021-12-31 22:00:00  금요일   맑음     대구광역시 달서구 장동   
39608  ACCIDENT_39608 2021-12-31 23:00:00  금요일   맑음     대구광역시 서구 비산동   

                  도로형태 노면상태  사고유형 사고유형 - 세부분류       법규위반  ... 피해운전자 상해정도 사망자수  \
0             단일로 - 기타   건조  차대사람  길가장자리구역통행중    안전운전불

In [94]:
for i in range(0, len(test_df)):
    address = test_df['시군구'][i]
    elements = address.split(" ")
    if len(elements) == 3:
        city = elements[0]
        county = elements[1]
        district = elements[2]
        test_df.at[i, '군'] = county
        test_df.at[i, '구'] = district
    else:
        print(f"잘못된 주소 형식입니다: {address}")

# 결과 출력
print(test_df)

                   ID                사고일시   요일 기상상태              시군구  \
0      ACCIDENT_39609 2022-01-01 01:00:00  토요일   맑음     대구광역시 수성구 상동   
1      ACCIDENT_39610 2022-01-01 01:00:00  토요일   맑음    대구광역시 수성구 지산동   
2      ACCIDENT_39611 2022-01-01 04:00:00  토요일   맑음  대구광역시 수성구 수성동2가   
3      ACCIDENT_39612 2022-01-01 04:00:00  토요일   맑음    대구광역시 수성구 신매동   
4      ACCIDENT_39613 2022-01-01 06:00:00  토요일   맑음    대구광역시 달서구 감삼동   
...               ...                 ...  ...  ...              ...   
10958  ACCIDENT_50567 2022-12-31 18:00:00  토요일   맑음     대구광역시 남구 대명동   
10959  ACCIDENT_50568 2022-12-31 18:00:00  토요일   맑음    대구광역시 수성구 시지동   
10960  ACCIDENT_50569 2022-12-31 20:00:00  토요일   맑음    대구광역시 수성구 연호동   
10961  ACCIDENT_50570 2022-12-31 20:00:00  토요일   맑음    대구광역시 수성구 범물동   
10962  ACCIDENT_50571 2022-12-31 21:00:00  토요일   맑음     대구광역시 동구 효목동   

              도로형태 노면상태  사고유형     x_coord    y_coord    군      구  
0       교차로 - 교차로안   건조  차대사람  128.615461  35.832127  수성구     상동  
1

# 외부 데이터 사용

이 부분은 이희원님의 코드를 참고하였습니다.

In [95]:
light_df = pd.read_csv('대구 보안등 정보.csv', encoding='cp949')[['설치개수', '소재지지번주소']]

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

light_df[['도시', '군', '구', '번지']] = light_df['소재지지번주소'].str.extract(location_pattern)
light_df = light_df.drop(columns=['소재지지번주소', '번지'])

light_df = light_df.groupby(['도시', '군', '구']).sum().reset_index()
light_df.reset_index(inplace=True, drop=True)

C:\Users\82105\AppData\Local\Temp\ipykernel_16576\3115058657.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.


In [96]:
parking_df = pd.read_csv('대구 주차장 정보.csv', encoding='cp949')[['소재지지번주소', '급지구분']]
parking_df = pd.get_dummies(parking_df, columns=['급지구분'])

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

parking_df[['도시', '군', '구', '번지']] = parking_df['소재지지번주소'].str.extract(location_pattern)
parking_df = parking_df.drop(columns=['소재지지번주소', '번지'])

parking_df = parking_df.groupby(['도시', '군', '구']).sum().reset_index()
parking_df.reset_index(inplace=True, drop=True)

In [97]:
child_area_df = pd.read_csv('대구 어린이 보호 구역 정보.csv', encoding='cp949').drop_duplicates()[['소재지지번주소']]
child_area_df['cnt'] = 1

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

child_area_df[['도시', '군', '구', '번지']] = child_area_df['소재지지번주소'].str.extract(location_pattern)
child_area_df = child_area_df.drop(columns=['소재지지번주소', '번지'])

child_area_df = child_area_df.groupby(['도시', '군', '구']).sum().reset_index()
child_area_df.reset_index(inplace=True, drop=True)

In [98]:
train_df = pd.merge(train_df, light_df, how='left', on=['군', '구'])
train_df = pd.merge(train_df, child_area_df, how='left', on=['군', '구'])
train_df = pd.merge(train_df, parking_df, how='left', on=['군', '구'])

test_df = pd.merge(test_df, light_df, how='left', on=['군', '구'])
test_df = pd.merge(test_df, child_area_df, how='left', on=['군', '구'])
test_df = pd.merge(test_df, parking_df, how='left', on=['군', '구'])

In [99]:
test_df['군'].unique()

array(['수성구', '달서구', '중구', '동구', '서구', '북구', '남구', '달성군'], dtype=object)

## One-hot-encoding

In [100]:
from sklearn.preprocessing import OneHotEncoder

# train_df와 test_df의 기상상태 및 노면상태 열 선택
train_categorical_data = train_df[['군']]
test_categorical_data = test_df[['군']]

# OneHotEncoder 인스턴스 생성 및 fit_transform 수행
encoder = OneHotEncoder()
train_encoded = encoder.fit_transform(train_categorical_data)
test_encoded = encoder.transform(test_categorical_data)

# OneHotEncoder가 사용한 카테고리 목록을 가져와서 카테고리 이름을 열 이름으로 변환
feature_names = encoder.get_feature_names_out(['군'])

# 밀집 행렬로 변환 (선택 사항)
train_encoded_dense = train_encoded.toarray()
test_encoded_dense = test_encoded.toarray()

# 데이터프레임으로 변환 (선택 사항)
import pandas as pd

train_encoded_df = pd.DataFrame(train_encoded_dense, columns=feature_names, index=train_df.index)
test_encoded_df = pd.DataFrame(test_encoded_dense, columns=feature_names, index=test_df.index)

# 기존 열 제거
train_df = train_df.drop(['군', ], axis=1)
test_df = test_df.drop(['군'], axis=1)

# 인코딩된 열 추가
train_df = pd.concat([train_df, train_encoded_df], axis=1)
test_df = pd.concat([test_df, test_encoded_df], axis=1)


In [101]:
train_df

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,급지구분_2,급지구분_3,군_남구,군_달서구,군_달성군,군_동구,군_북구,군_서구,군_수성구,군_중구
0,ACCIDENT_00000,2019-01-01 00:00:00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,ACCIDENT_00001,2019-01-01 00:00:00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ACCIDENT_00002,2019-01-01 01:00:00,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,ACCIDENT_00003,2019-01-01 02:00:00,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,9.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,ACCIDENT_00004,2019-01-01 04:00:00,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39604,2021-12-31 19:00:00,금요일,맑음,대구광역시 수성구 수성동3가,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
39605,ACCIDENT_39605,2021-12-31 19:00:00,금요일,맑음,대구광역시 달서구 상인동,단일로 - 기타,건조,차대차,측면충돌,안전거리미확보,...,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
39606,ACCIDENT_39606,2021-12-31 21:00:00,금요일,맑음,대구광역시 달서구 월성동,교차로 - 교차로안,건조,차대차,측면충돌,교차로운행방법위반,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
39607,ACCIDENT_39607,2021-12-31 22:00:00,금요일,맑음,대구광역시 달서구 장동,기타 - 기타,건조,차대차,추돌,안전운전불이행,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


# 시군구별 발생횟수 feature 추가

In [102]:
accident_counts = train_df['시군구'].value_counts().reset_index()
accident_counts.columns = ['시군구', '사고횟수']

In [103]:
# '시군구'별 사고 횟수를 train_df에 매핑하기 위해 '시군구' 컬럼을 기준으로 사고 횟수를 가져옵니다.
train_df['사고발생횟수'] = train_df['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])


In [104]:
# '시군구'별 사고 횟수를 train_df에 매핑하기 위해 '시군구' 컬럼을 기준으로 사고 횟수를 가져옵니다.
test_df['사고발생횟수'] = test_df['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])


In [105]:
train_df

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,급지구분_3,군_남구,군_달서구,군_달성군,군_동구,군_북구,군_서구,군_수성구,군_중구,사고발생횟수
0,ACCIDENT_00000,2019-01-01 00:00:00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,131
1,ACCIDENT_00001,2019-01-01 00:00:00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,678
2,ACCIDENT_00002,2019-01-01 01:00:00,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,604
3,ACCIDENT_00003,2019-01-01 02:00:00,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,426
4,ACCIDENT_00004,2019-01-01 04:00:00,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39604,2021-12-31 19:00:00,금요일,맑음,대구광역시 수성구 수성동3가,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,104
39605,ACCIDENT_39605,2021-12-31 19:00:00,금요일,맑음,대구광역시 달서구 상인동,단일로 - 기타,건조,차대차,측면충돌,안전거리미확보,...,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1143
39606,ACCIDENT_39606,2021-12-31 21:00:00,금요일,맑음,대구광역시 달서구 월성동,교차로 - 교차로안,건조,차대차,측면충돌,교차로운행방법위반,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,618
39607,ACCIDENT_39607,2021-12-31 22:00:00,금요일,맑음,대구광역시 달서구 장동,기타 - 기타,건조,차대차,추돌,안전운전불이행,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,125


In [106]:
test_df

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,x_coord,y_coord,...,급지구분_3,군_남구,군_달서구,군_달성군,군_동구,군_북구,군_서구,군_수성구,군_중구,사고발생횟수
0,ACCIDENT_39609,2022-01-01 01:00:00,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람,128.615461,35.832127,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,338
1,ACCIDENT_39610,2022-01-01 01:00:00,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람,128.634391,35.819590,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,492
2,ACCIDENT_39611,2022-01-01 04:00:00,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차,128.615301,35.858071,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,147
3,ACCIDENT_39612,2022-01-01 04:00:00,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차,128.710784,35.839630,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,372
4,ACCIDENT_39613,2022-01-01 06:00:00,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차,128.538134,35.845468,...,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,ACCIDENT_50567,2022-12-31 18:00:00,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차,128.561332,35.841187,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1749
10959,ACCIDENT_50568,2022-12-31 18:00:00,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차,128.688102,35.843276,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,167
10960,ACCIDENT_50569,2022-12-31 20:00:00,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차,128.672022,35.849466,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,107
10961,ACCIDENT_50570,2022-12-31 20:00:00,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차,128.643392,35.815755,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,239


In [107]:
# datetime 컬럼 처리
train_df['사고일시'] = pd.to_datetime(train_df['사고일시'])
test_df['사고일시'] = pd.to_datetime(test_df['사고일시'])

# datetime을 여러 파생 변수로 변환
for df in [train_df, test_df]:
    df['year'] = df['사고일시'].dt.year
    df['month'] = df['사고일시'].dt.month
    df['day'] = df['사고일시'].dt.day
    df['hour'] = df['사고일시'].dt.hour
    df['minute'] = df['사고일시'].dt.minute
    df['weekday'] = df['사고일시'].dt.weekday

In [108]:
holi_weekday = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01']

In [109]:
train_df['사고일시'] = pd.to_datetime(train_df['사고일시'])
train_df['day_of_week'] = train_df['사고일시'].dt.dayofweek
train_df['holiday'] = np.where((train_df.day_of_week >= 5) | (train_df.사고일시.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)

In [110]:
test_df['사고일시'] = pd.to_datetime(test_df['사고일시'])
test_df['day_of_week'] = test_df['사고일시'].dt.dayofweek
test_df['holiday'] = np.where((test_df.day_of_week >= 5) | (test_df.사고일시.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)

In [111]:
train_df = train_df.drop(columns=['사고유형 - 세부분류','법규위반','가해운전자 차종','가해운전자 성별','가해운전자 연령',
                               '가해운전자 상해정도','사망자수','사망자수','중상자수','피해운전자 차종','피해운전자 성별',
                      '피해운전자 연령','피해운전자 상해정도','경상자수','부상자수','사고유형 - 세부분류'])

# 계절 변수 추가

In [112]:
def group_season(df):
    df.loc[(df['month'] == 3) | (df['month'] == 4) | (df['month'] == 5), 'season'] = '봄'
    df.loc[(df['month'] == 6) | (df['month'] == 7) | (df['month'] == 8), 'season'] = '여름'
    df.loc[(df['month'] == 9) | (df['month'] == 10) | (df['month'] == 11), 'season'] = '가을'
    df.loc[(df['month'] == 12) | (df['month'] == 1) | (df['month'] == 2), 'season'] = '겨울'
    return df['season']

train_df['Cosine_Time'] = np.cos(2 * np.pi * train_df['hour'] / 24)
test_df['Cosine_Time'] = np.cos(2 * np.pi * test_df['hour'] / 24)
train_df['season'] = group_season(train_df)
test_df['season'] = group_season(test_df)

# Target-Encoding

In [113]:
# Target encoding
from category_encoders.target_encoder import TargetEncoder

categorical_features = ['ID','시군구','요일','도로형태','사고유형','구','기상상태', '노면상태','season']

for i in categorical_features:
    tr_encoder = TargetEncoder(cols=[i])
    train_df[i] = tr_encoder.fit_transform(train_df[i], train_df['ECLO'])
    test_df[i] = tr_encoder.transform(test_df[i])

In [114]:
target = train_df['ECLO']

In [115]:
train_df['ECLO'] = target

In [116]:
train_df.corr()

C:\Users\82105\AppData\Local\Temp\ipykernel_16576\1233823885.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.


,ID,요일,기상상태,시군구,도로형태,노면상태,사고유형,ECLO,x_coord,y_coord,...,year,month,day,hour,minute,weekday,day_of_week,holiday,Cosine_Time,season
ID,1.000000,0.061771,0.014750,0.118496,0.086154,0.018617,0.137211,1.000000,-0.001044,-0.000636,...,-0.031281,-0.006871,-0.012303,-0.017355,NaN,0.039407,0.039407,0.060091,0.037052,0.013063
요일,0.061771,1.000000,0.005726,-0.011276,0.009456,0.003387,0.010758,0.061771,0.012584,0.014245,...,-0.015829,-0.000869,-0.001398,-0.028288,NaN,0.637956,0.637956,0.861151,0.031918,0.010607
기상상태,0.014750,0.005726,1.000000,0.006730,0.005441,0.811933,-0.019851,0.014750,-0.005946,0.015568,...,0.028432,-0.019630,-0.012132,0.025395,NaN,-0.007867,-0.007867,0.004480,0.099668,0.019916
시군구,0.118496,-0.011276,0.006730,1.000000,0.025979,-0.005277,0.093599,0.118496,-0.032962,-0.047109,...,0.013896,0.011130,0.001173,-0.014139,NaN,-0.010625,-0.010625,-0.009383,-0.046177,0.007512
도로형태,0.086154,0.009456,0.005441,0.025979,1.000000,0.019300,0.238355,0.086154,0.008873,0.032401,...,-0.008411,-0.002795,0.006277,-0.014810,NaN,0.008161,0.008161,0.007643,0.022999,-0.002123
노면상태,0.018617,0.003387,0.811933,-0.005277,0.019300,1.000000,-0.008089,0.018617,0.013216,0.029786,...,0.018757,-0.023078,-0.017716,0.019702,NaN,-0.013314,-0.013314,0.000965,0.081766,0.016906
사고유형,0.137211,0.010758,-0.019851,0.093599,0.238355,-0.008089,1.000000,0.137211,-0.020236,-0.007159,...,0.008337,0.000300,-0.000257,0.009275,NaN,0.009591,0.009591,0.008729,-0.042967,0.001176
ECLO,1.000000,0.061771,0.014750,0.118496,0.086154,0.018617,0.137211,1.000000,-0.001044,-0.000636,...,-0.031281,-0.006871,-0.012303,-0.017355,NaN,0.039407,0.039407,0.060091,0.037052,0.013063
x_coord,-0.001044,0.012584,-0.005946,-0.032962,0.008873,0.013216,-0.020236,-0.001044,1.000000,0.350552,...,0.003080,0.003531,0.000219,-0.003681,NaN,0.008033,0.008033,0.014505,-0.019494,-0.000372
y_coord,-0.000636,0.014245,0.015568,-0.047109,0.032401,0.029786,-0.007159,-0.000636,0.350552,1.000000,...,0.003999,-0.009916,-0.000462,0.001029,NaN,0.007699,0.007699,0.012187,-0.015556,0.000277


In [117]:
#train_df = train_df.drop(columns=['ECLO'])

In [118]:
#test_df = test_df.drop(columns=['사고일시'])

In [119]:
train_df.columns

Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', 'ECLO',
       'x_coord', 'y_coord', '구', '도시_x', '설치개수', '도시_y', 'cnt', '도시',
       '급지구분_1', '급지구분_2', '급지구분_3', '군_남구', '군_달서구', '군_달성군', '군_동구', '군_북구',
       '군_서구', '군_수성구', '군_중구', '사고발생횟수', 'year', 'month', 'day', 'hour',
       'minute', 'weekday', 'day_of_week', 'holiday', 'Cosine_Time', 'season'],
      dtype='object')

## 비비드백님의 인사이트를 참고하였습니다

In [124]:
test_x = test_df.drop(columns=['ID','도시_x','도시_y','도시','year','month','day','hour','minute','day_of_week','사고일시']).copy()
train_x = train_df[test_x.columns].copy()
train_y = train_df['ECLO'].copy()

In [125]:
test_x.columns

Index(['요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', 'x_coord', 'y_coord', '구',
       '설치개수', 'cnt', '급지구분_1', '급지구분_2', '급지구분_3', '군_남구', '군_달서구', '군_달성군',
       '군_동구', '군_북구', '군_서구', '군_수성구', '군_중구', '사고발생횟수', 'weekday', 'holiday',
       'Cosine_Time', 'season'],
      dtype='object')

In [126]:
test_x

,요일,기상상태,시군구,도로형태,노면상태,사고유형,x_coord,y_coord,구,설치개수,...,군_동구,군_북구,군_서구,군_수성구,군_중구,사고발생횟수,weekday,holiday,Cosine_Time,season
0,4.920811,4.712888,4.881657,5.006142,4.712878,3.817650,128.615461,35.832127,4.881657,700.0,...,0.0,0.0,0.0,1.0,0.0,338,5,1,9.659258e-01,4.659111
1,4.920811,4.712888,4.563008,4.653990,4.712878,3.817650,128.634391,35.819590,4.563008,NaN,...,0.0,0.0,0.0,1.0,0.0,492,5,1,9.659258e-01,4.659111
2,4.920811,4.712888,4.945578,5.006142,4.712878,4.944597,128.615301,35.858071,4.945578,NaN,...,0.0,0.0,0.0,1.0,0.0,147,5,1,5.000000e-01,4.659111
3,4.920811,4.712888,4.438172,4.653990,4.712878,4.944597,128.710784,35.839630,4.438172,NaN,...,0.0,0.0,0.0,1.0,0.0,372,5,1,5.000000e-01,4.659111
4,4.920811,4.712888,4.738938,5.006142,4.712878,4.944597,128.538134,35.845468,4.738938,932.0,...,0.0,0.0,0.0,0.0,0.0,678,5,1,6.123234e-17,4.659111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,4.920811,4.712888,4.492281,6.555670,4.712878,4.944597,128.561332,35.841187,4.492281,5377.0,...,0.0,0.0,0.0,0.0,0.0,1749,5,1,-1.836970e-16,4.659111
10959,4.920811,4.712888,4.928144,4.653990,4.712878,4.944597,128.688102,35.843276,4.928144,NaN,...,0.0,0.0,0.0,1.0,0.0,167,5,1,-1.836970e-16,4.659111
10960,4.920811,4.712888,5.355036,4.653990,4.712878,4.944597,128.672022,35.849466,5.355036,NaN,...,0.0,0.0,0.0,1.0,0.0,107,5,1,5.000000e-01,4.659111
10961,4.920811,4.712888,5.079498,4.863035,4.712878,4.944597,128.643392,35.815755,5.079498,NaN,...,0.0,0.0,0.0,1.0,0.0,239,5,1,5.000000e-01,4.659111


In [37]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

### AUTOML

In [40]:
import matplotlib
matplotlib.use('Agg')

In [127]:
from supervised.automl import AutoML
automl = AutoML(mode="Compete",
                algorithms = ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost'],
                n_jobs = -1,total_time_limit=43200, eval_metric="rmse", ml_task = "regression",)
#3.209155

In [128]:
automl.fit(train_x, train_y)

AutoML directory: AutoML_10
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree rmse 3.224312 trained in 1.55 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 4 models
1_Default_LightGBM rmse 3.236253 trained in 50.72 seconds
2_Default_Xgboost rmse 3.233645 trained in 50.51 seconds
3_Default_CatBoost rmse 3.230494 trained in 45.14 seconds
4_Default_Ran

D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value 

3_Default_CatBoost_KMeansFeatures rmse 3.231496 trained in 21.63 seconds


D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value 

31_CatBoost_KMeansFeatures rmse 3.233175 trained in 44.49 seconds


D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
D:\mdrive\dacon\envs\traffic\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value 

14_LightGBM_KMeansFeatures rmse 3.232422 trained in 18.65 seconds
* Step insert_random_feature will try to check up to 1 model
31_CatBoost_GoldenFeatures_RandomFeature rmse 3.23167 trained in 31.49 seconds


C:\Users\82105\AppData\Roaming\Python\Python38\site-packages\supervised\utils\additional_metrics.py:701: UserWarning: Glyph 46020 (\N{HANGUL SYLLABLE DO}) missing from current font.
C:\Users\82105\AppData\Roaming\Python\Python38\site-packages\supervised\utils\additional_metrics.py:701: UserWarning: Glyph 47196 (\N{HANGUL SYLLABLE RO}) missing from current font.
C:\Users\82105\AppData\Roaming\Python\Python38\site-packages\supervised\utils\additional_metrics.py:701: UserWarning: Glyph 54805 (\N{HANGUL SYLLABLE HYEONG}) missing from current font.
C:\Users\82105\AppData\Roaming\Python\Python38\site-packages\supervised\utils\additional_metrics.py:701: UserWarning: Glyph 53468 (\N{HANGUL SYLLABLE TAE}) missing from current font.
C:\Users\82105\AppData\Roaming\Python\Python38\site-packages\supervised\utils\additional_metrics.py:701: UserWarning: Glyph 49884 (\N{HANGUL SYLLABLE SI}) missing from current font.
C:\Users\82105\AppData\Roaming\Python\Python38\site-packages\supervised\utils\additio

Drop features ['설치개수', '군_중구', '군_달서구', '급지구분_2', '군_북구', '군_동구', '군_달성군', '급지구분_3', '군_수성구', 'random_feature', '군_서구', 'x_coord']
* Step features_selection will try to check up to 4 models
31_CatBoost_GoldenFeatures_SelectedFeatures rmse 3.228827 trained in 27.08 seconds
14_LightGBM_GoldenFeatures_SelectedFeatures rmse 3.230488 trained in 7.82 seconds
37_RandomForest_SelectedFeatures rmse 3.23041 trained in 29.31 seconds
10_Xgboost_SelectedFeatures rmse 3.230996 trained in 7.91 seconds
* Step hill_climbing_1 will try to check up to 16 models
41_CatBoost_GoldenFeatures_SelectedFeatures rmse 3.408589 trained in 59.5 seconds
42_CatBoost_GoldenFeatures rmse 3.408944 trained in 63.53 seconds
43_RandomForest_SelectedFeatures rmse 3.231712 trained in 23.86 seconds
44_CatBoost_GoldenFeatures rmse 3.413952 trained in 20.17 seconds
45_LightGBM_GoldenFeatures_SelectedFeatures rmse 3.231205 trained in 9.66 seconds
46_LightGBM_GoldenFeatures_SelectedFeatures rmse 3.230325 trained in 8.64 seconds
4

AutoML(algorithms=['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost'],
       eval_metric='rmse', ml_task='regression', mode='Compete',
       total_time_limit=43200)

In [132]:
pred = automl.predict(test_x)

## 제출

In [133]:
submission = pd.read_csv('sample_submission.csv')

In [134]:
submission['ECLO'] = pred

In [39]:
submission.loc[ submission['ECLO'] < 0.0, 'ECLO'] = 0.0

In [135]:
min(submission['ECLO'])

2.668001039299015

In [137]:
submission.to_csv('submit.csv', index=False)

In [136]:
submission

,ID,ECLO
0,ACCIDENT_39609,3.778110
1,ACCIDENT_39610,3.374648
2,ACCIDENT_39611,4.682658
3,ACCIDENT_39612,4.127119
4,ACCIDENT_39613,4.441856
...,...,...
10958,ACCIDENT_50567,4.662175
10959,ACCIDENT_50568,4.389722
10960,ACCIDENT_50569,4.756258
10961,ACCIDENT_50570,4.650978
